In [7]:
import requests
import mysql.connector
from mysql.connector import errorcode
import time
import discord_notifier


# Function to insert a batch of summoner data into the database
def insert_summoner_data_batch(summoner_data_batch):
    try:
        cnx = mysql.connector.connect(user='lol_user', password='your_password',
                                      host='35.203.46.85', database='lol_retention')
        cursor = cnx.cursor()
        
        add_summoner = """
        INSERT INTO summoner_details (summoner_id, account_id, puuid, profile_icon_id, revision_date, summoner_level)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            profile_icon_id = VALUES(profile_icon_id), revision_date = VALUES(revision_date), 
            summoner_level = VALUES(summoner_level)
        """
        
        cursor.executemany(add_summoner, summoner_data_batch)
        cnx.commit()
        cursor.close()
        cnx.close()
    except mysql.connector.Error as err:
        print("Error:", err)

# Function to pull summoner data from the API and insert it into the database
def pull_summoner_data():
    api_key = "RGAPI-39403bc3-0460-4e9a-b6eb-ef92e74fabd0"
    base_url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/"
    summoner_data_batch = []

    # Connect to the database to get all summoner IDs
    try:
        cnx = mysql.connector.connect(user='lol_user', password='your_password',
                                      host='35.203.46.85', database='lol_retention')
        cursor = cnx.cursor()
        # Query to find summoner_ids in entries table that don't have corresponding records in summoner_details
        cursor.execute("""
            SELECT e.summoner_id
            FROM entries e
            WHERE NOT EXISTS (
                SELECT 1
                FROM summoner_details sd
                WHERE e.summoner_id = sd.summoner_id
            );
        """)
        summoner_ids = cursor.fetchall()
        cursor.close()
        cnx.close()

    except mysql.connector.Error as err:
        print("Error:", err)
        return

    print(f"values that need to be pull form the Riot API: {len(summoner_ids)}")

    # Iterate over summoner IDs and pull data one by one
    val = 1
    for summoner_id in summoner_ids:
        
        print(f"fetching values for: {summoner_id} which is in position: {val}")
        url = f"{base_url}{summoner_id[0]}?api_key={api_key}"
        response = requests.get(url)
        if response.status_code != 200:
            print("Error with the request:", response.status_code)
            continue
        
        summoner_data = response.json()
        summoner_data_batch.append((
            summoner_data['id'], summoner_data['accountId'], summoner_data['puuid'], 
            summoner_data['profileIconId'], summoner_data['revisionDate'], summoner_data['summonerLevel']
        ))

        # Respect the rate limit
        val += 1
        time.sleep(1.3)

    # Insert all fetched data into the database in a single batch
    if summoner_data_batch:
        print('inserting vals into db')
        insert_summoner_data_batch(summoner_data_batch)

if __name__ == "__main__":
    try:
        pull_summoner_data()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        discord_notifier.send_discord_message()
    

values that need to be pull form the Riot API: 3919
fetching values for: ('__jjqHjcwKo5a0aE8kKG0YQm-YSBJJ6Y0nkiG7tY6VXb5nNL',) which is in position: 1
fetching values for: ('_-W8LreB2aP7IWEr8ZNkZ-DoxmPpElvtgrB5QDwwX1MupWnf',) which is in position: 2
fetching values for: ('_-WAd8MlReczz7r0hx_W6v6o4ay_W2auXaTcvmM3qrJ8rg9R',) which is in position: 3
fetching values for: ('_0eVSJq9KObnuAp3eEt7tSMZH2i8fjpQ6g9y0fQTtockxmPz',) which is in position: 4
fetching values for: ('_1wCXrgVu40feJj3q_VDBrnBrE06h8WC4DG6FKwtl9CrSEl1',) which is in position: 5
fetching values for: ('_30aBipvO3o9AyGQf0pn1Q8I_ZS20-YlYjgFo0TMnEnH0rJ9',) which is in position: 6
fetching values for: ('_3EMatKLCIBHjIoW-UUMFvIJSuIvGTb_bkMn55ftuw02KiPa',) which is in position: 7
fetching values for: ('_5BZMxFgvJPHT5O3d3OpemrqsElWZ3o1W20kJzvC1CPL2k2k',) which is in position: 8
fetching values for: ('_6ffDMBbdoDFUPSInOM1f9EpYYvEP502-q7J9X6bd5YimM2H',) which is in position: 9
fetching values for: ('_8D10NuVA1UOdbEQKQtiIqORJIQ8PlwB57